# Quantium Virtual Internship Project - Retail Strategy and Analytics

## Task 2. Experimentation and Uplift Testing

In [1]:
# make all the necessery imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import datetime as dt
import re
import numpy as np
from scipy.stats import ttest_ind

%matplotlib inline

### Import datasets as pandas dataframes

In [2]:
task2_data = pd.read_csv("task2_data.csv")

In [3]:
task2_data.head()

LYLTY_CARD_NBR        DATE  STORE_NBR  TXN_ID  PROD_NBR  \
0            1000  2018-10-17          1       1         5   
1            1002  2018-09-16          1       2        58   
2            1003  2019-03-07          1       3        52   
3            1003  2019-03-08          1       4       106   
4            1004  2018-11-02          1       5        96   

                                PROD_NAME  PROD_QTY  TOT_SALES  PACK_SIZE  \
0  Natural Chip        Compny SeaSalt175g         2        6.0        175   
1   Red Rock Deli Chikn&Garlic Aioli 150g         1        2.7        150   
2   Grain Waves Sour    Cream&Chives 210G         1        3.6        210   
3  Natural ChipCo      Hony Soy Chckn175g         1        3.0        175   
4          WW Original Stacked Chips 160g         1        1.9        160   

        BRAND              LIFESTAGE PREMIUM_CUSTOMER  
0     NATURAL  YOUNG SINGLES/COUPLES          Premium  
1         RRD  YOUNG SINGLES/COUPLES       Mainstream  
2     GRNWVES         YOUNG FAMILIES           Budget  
3     NATURAL         YOUNG FAMILIES           Budget  
4  WOOLWORTHS  OLDER SINGLES/COUPLES       Mainstream

### Calculate measures of interest and filter stores that are present throughout the pre-trial period.

In [5]:
# Create a yyyymm column
task2_data["MONTH"] = (task2_data["DATE"].str[:4] + task2_data["DATE"].str[5:7]).astype(int)
task2_data["MONTH"].head()

0    201810
1    201809
2    201903
3    201903
4    201811
Name: MONTH, dtype: int64

In [6]:
# Create a new dataframe grouped by store number and month with desired aggregates

stores_by_month = task2_data.groupby(['STORE_NBR', 'MONTH']).agg({
    'TOT_SALES': 'sum', 'LYLTY_CARD_NBR': pd.Series.nunique,
'TXN_ID': 'count', 'PROD_QTY': 'sum'}).reset_index()

stores_by_month.head()

STORE_NBR   MONTH  TOT_SALES  LYLTY_CARD_NBR  TXN_ID  PROD_QTY
0          1  201807      206.9              49      52        62
1          1  201808      176.1              42      43        54
2          1  201809      278.8              59      62        75
3          1  201810      188.1              44      45        58
4          1  201811      192.6              46      47        57

In [7]:
#rename the columns to represent their real meaning

stores_by_month = stores_by_month.rename(columns= {'LYLTY_CARD_NBR': 'NBR_CUSTOMERS',
                                'TXN_ID': 'NBR_TRANSACTIONS'})

stores_by_month.columns

Index(['STORE_NBR', 'MONTH', 'TOT_SALES', 'NBR_CUSTOMERS', 'NBR_TRANSACTIONS',
       'PROD_QTY'],
      dtype='object')

In [8]:
# Calculate transactions per customer, chips per transaction, chips per customer and average price per unit

stores_by_month['TXN_PR_CUST'] = (stores_by_month['NBR_TRANSACTIONS']/stores_by_month['NBR_CUSTOMERS']).round(2)
stores_by_month['CHIPS_PR_TXN'] = (stores_by_month['PROD_QTY']/stores_by_month['NBR_TRANSACTIONS']).round(2)
stores_by_month['CHIPS_PR_CUST'] = (stores_by_month['PROD_QTY']/stores_by_month['NBR_CUSTOMERS']).round(2)
stores_by_month['AVG_PRICE_PR_UNIT'] = (stores_by_month['TOT_SALES']/stores_by_month['PROD_QTY']).round(2)

stores_by_month.head()

STORE_NBR   MONTH  TOT_SALES  NBR_CUSTOMERS  NBR_TRANSACTIONS  PROD_QTY  \
0          1  201807      206.9             49                52        62   
1          1  201808      176.1             42                43        54   
2          1  201809      278.8             59                62        75   
3          1  201810      188.1             44                45        58   
4          1  201811      192.6             46                47        57   

   TXN_PR_CUST  CHIPS_PR_TXN  CHIPS_PR_CUST  AVG_PRICE_PR_UNIT  
0         1.06          1.19           1.27               3.34  
1         1.02          1.26           1.29               3.26  
2         1.05          1.21           1.27               3.72  
3         1.02          1.29           1.32               3.24  
4         1.02          1.21           1.24               3.38

In [39]:
# Get the stores with full observation periods
full_obs = stores_by_month.groupby(['STORE_NBR']).agg({'MONTH': pd.Series.nunique}).reset_index()
full_obs = test[test['MONTH'] == 12]
full_obs_stores = full_obs['STORE_NBR'].unique()
full_obs_stores


# Filter the stores with full observation periods 
stores_by_month = stores_by_month[stores_by_month.STORE_NBR.isin(full_obs_stores)]

# Get the data for pre trial period
pre_trial_data = stores_by_month[stores_by_month['MONTH'] < 201902]

### Ranking how similar each potential control store and identifying the most similar control stores for each trial store.

In [60]:
def calcCorrTable(metricCol, storeComparison, inputTable=pre_trial_data):
        """Calculate correlation for a measure, looping through each control store.
    Args:
        metricCol (str): Name of column containing store's metric to perform correlation test on.
        storeComparison (int): Trial store's number.
        inputTable (dataframe):  Metric table with potential comparison stores.
        
    Returns:
        DataFrame: Monthly correlation table between Trial and each Control stores.
    """
    control_store_nbrs = inputTable[~inputTable["STORE_NBR"].isin([77, 86, 88])]["STORE_NBR"].unique()
    corrs = pd.DataFrame(columns = ["MONTH", "Trial_Str", "Ctrl_Str", "Corr_Score"])
    trial_store = inputTable[inputTable["STORE_NBR"] == storeComparison][metricCol].reset_index()
    for control in control_store_nbrs:
        concat_df = pd.DataFrame(columns = ["MONTH", "Trial_Str", "Ctrl_Str", "Corr_Score"])
        control_store = inputTable[inputTable["STORE_NBR"] == control][metricCol].reset_index()
        concat_df["Corr_Score"] = trial_store.corrwith(control_store, axis=1)
        concat_df["Trial_Str"] = storeComparison
        concat_df["Ctrl_Str"] = control
        concat_df["MONTH"] = list(inputTable[inputTable["STORE_NBR"] == storeComparison]["MONTH"])
        corrs = pd.concat([corrs, concat_df])
    return corrs

In [61]:
corr_table = pd.DataFrame()
for trial_num in [77, 86, 88]:
    corr_table = pd.concat([corr_table, calcCorrTable(["TOT_SALES", "NBR_CUSTOMERS", "TXN_PR_CUST", "CHIPS_PR_TXN", "AVG_PRICE_PR_UNIT"], trial_num)])
    
corr_table.head(8)

MONTH Trial_Str Ctrl_Str  Corr_Score
0  201807        77        1    0.070545
1  201808        77        1    0.027335
2  201809        77        1    0.002469
3  201810        77        1   -0.019985
4  201811        77        1    0.030101
5  201812        77        1    0.064175
6  201901        77        1    0.001572
0  201807        77        2    0.143091

In [62]:
def calculateMagnitudeDistance(metricCol, storeComparison, inputTable=pre_trial_data):
    """Calculate standardised magnitude distance for a measure, looping through each control store.
    Args:
        metricCol (str): Name of column containing store's metric to perform distance calculation on.
        storeComparison (int): Trial store's number.
        inputTable (dataframe):  Metric table with potential comparison stores.
        
    Returns:
        DataFrame: Monthly magnitude-distance table between Trial and each Control stores.
    """
    control_store_nbrs = inputTable[~inputTable["STORE_NBR"].isin([77, 86, 88])]["STORE_NBR"].unique()
    dists = pd.DataFrame()
    trial_store = inputTable[inputTable["STORE_NBR"] == storeComparison][metricCol]
    for control in control_store_nbrs:
        concat_df  = abs(inputTable[inputTable["STORE_NBR"] == storeComparison].reset_index()[metricCol] - inputTable[inputTable["STORE_NBR"] == control].reset_index()[metricCol])
        concat_df["MONTH"] = list(inputTable[inputTable["STORE_NBR"] == storeComparison]["MONTH"])
        concat_df["Trial_Str"] = storeComparison
        concat_df["Ctrl_Str"] = control
        dists = pd.concat([dists, concat_df])
    for col in metricCol:
        dists[col] = 1 - ((dists[col] - dists[col].min()) / (dists[col].max() - dists[col].min()))
    dists["magnitude"] = dists[metricCol].mean(axis=1)
    return dists


In [63]:
dist_table = pd.DataFrame()
for trial_num in [77, 86, 88]:
    dist_table = pd.concat([dist_table, calculateMagnitudeDistance(["TOT_SALES", "NBR_CUSTOMERS", "TXN_PR_CUST", "CHIPS_PR_TXN", "AVG_PRICE_PR_UNIT"], trial_num)])
    
dist_table.head(8)
dist_table


TOT_SALES  NBR_CUSTOMERS  TXN_PR_CUST  CHIPS_PR_TXN  AVG_PRICE_PR_UNIT  \
0    0.935431       0.980769     0.951220      0.492537           0.887574   
1    0.942972       0.951923     1.000000      0.582090           0.887574   
2    0.961503       0.836538     1.000000      0.432836           0.704142   
3    0.988221       0.932692     0.975610      0.880597           0.591716   
4    0.962149       0.951923     0.878049      0.537313           0.834320   
..        ...            ...          ...           ...                ...   
2    0.207554       0.286822     0.466667      0.970000           0.925311   
3    0.346797       0.387597     0.600000      0.940000           0.970954   
4    0.286706       0.310078     0.633333      0.930000           0.966805   
5    0.347151       0.387597     0.400000      0.890000           0.962656   
6    0.402353       0.449612     0.466667      0.920000           0.970954   

     MONTH  Trial_Str  Ctrl_Str  magnitude  
0   201807         77         1   0.849506  
1   201808         77         1   0.872912  
2   201809         77         1   0.787004  
3   201810         77         1   0.873767  
4   201811         77         1   0.832751  
..     ...        ...       ...        ...  
2   201809         88       272   0.571271  
3   201810         88       272   0.649070  
4   201811         88       272   0.625384  
5   201812         88       272   0.597481  
6   201901         88       272   0.641917  

[5397 rows x 9 columns]

In [65]:
def combine_corr_dist(metricCol, storeComparison, inputTable=pre_trial_data):
    corrs = calcCorrTable(metricCol, storeComparison, inputTable)
    dists = calculateMagnitudeDistance(metricCol, storeComparison, inputTable)
    dists = dists.drop(metricCol, axis=1)
    combine = pd.merge(corrs, dists, on=["MONTH", "Trial_Str", "Ctrl_Str"])
    return combine

In [66]:
compare_metrics_table1 = pd.DataFrame()
for trial_num in [77, 86, 88]:
    compare_metrics_table1 = pd.concat([compare_metrics_table1, combine_corr_dist(["TOT_SALES"], trial_num)])

In [67]:
corr_weight = 0.5
dist_weight = 1 - corr_weight

In [68]:
#Top 5 highest Composite Score for each Trial Store based on TOT_SALES
grouped_comparison_table1 = compare_metrics_table1.groupby(["Trial_Str", "Ctrl_Str"]).mean().reset_index()
grouped_comparison_table1["CompScore"] = (corr_weight * grouped_comparison_table1["Corr_Score"]) + (dist_weight * grouped_comparison_table1["magnitude"])
for trial_num in compare_metrics_table1["Trial_Str"].unique():
    print(grouped_comparison_table1[grouped_comparison_table1["Trial_Str"] == trial_num].sort_values(ascending=False, by="CompScore").head(), '\n')

     Trial_Str  Ctrl_Str  Corr_Score  magnitude  CompScore
218         77       233         1.0   0.986477   0.993238
239         77       255         1.0   0.979479   0.989739
177         77       188         1.0   0.977663   0.988831
49          77        53         1.0   0.976678   0.988339
120         77       131         1.0   0.976267   0.988134 

     Trial_Str  Ctrl_Str  Corr_Score  magnitude  CompScore
356         86       109         1.0   0.966783   0.983391
401         86       155         1.0   0.965876   0.982938
464         86       222         1.0   0.962280   0.981140
467         86       225         1.0   0.960512   0.980256
471         86       229         1.0   0.951704   0.975852 

     Trial_Str  Ctrl_Str  Corr_Score  magnitude  CompScore
551         88        40         1.0   0.941165   0.970582
538         88        26         1.0   0.904377   0.952189
582         88        72         1.0   0.903800   0.951900
517         88         4         1.0   0.903466   0.

In [72]:
compare_metrics_table2 = pd.DataFrame()
for trial_num in [77, 86, 88]:
    compare_metrics_table2 = pd.concat([compare_metrics_table2, combine_corr_dist(["NBR_CUSTOMERS"], trial_num)])

In [74]:
#Top 5 highest Composite Score for each Trial Store based on nCustomers
grouped_comparison_table2 = compare_metrics_table2.groupby(["Trial_Str", "Ctrl_Str"]).mean().reset_index()
grouped_comparison_table2["CompScore"] = (corr_weight * grouped_comparison_table2["Corr_Score"]) + (dist_weight * grouped_comparison_table2["magnitude"])
for trial_num in compare_metrics_table2["Trial_Str"].unique():
    print(grouped_comparison_table2[grouped_comparison_table2["Trial_Str"] == trial_num].sort_values(ascending=False, by="CompScore").head(), '\n')

     Trial_Str  Ctrl_Str  Corr_Score  magnitude  CompScore
218         77       233         1.0   0.993132   0.996566
38          77        41         1.0   0.976648   0.988324
101         77       111         1.0   0.968407   0.984203
105         77       115         1.0   0.967033   0.983516
15          77        17         1.0   0.965659   0.982830 

     Trial_Str  Ctrl_Str  Corr_Score  magnitude  CompScore
401         86       155         1.0   0.986772   0.993386
467         86       225         1.0   0.969577   0.984788
356         86       109         1.0   0.969577   0.984788
471         86       229         1.0   0.964286   0.982143
293         86        39         1.0   0.961640   0.980820 

     Trial_Str  Ctrl_Str  Corr_Score  magnitude  CompScore
736         88       237         1.0   0.987818   0.993909
705         88       203         1.0   0.944629   0.972315
551         88        40         1.0   0.942414   0.971207
668         88       165         1.0   0.935770   0.

In [75]:
for trial_num in compare_metrics_table2["Trial_Str"].unique():
    a = grouped_comparison_table1[grouped_comparison_table1["Trial_Str"] == trial_num].sort_values(ascending=False, by="CompScore").set_index(["Trial_Str", "Ctrl_Str"])["CompScore"]
    b = grouped_comparison_table2[grouped_comparison_table2["Trial_Str"] == trial_num].sort_values(ascending=False, by="CompScore").set_index(["Trial_Str", "Ctrl_Str"])["CompScore"]
    print((pd.concat([a,b], axis=1).sum(axis=1)/2).sort_values(ascending=False).head(3), '\n')

Trial_Str  Ctrl_Str
77         233         0.994902
           41          0.986020
           46          0.984762
dtype: float64 

Trial_Str  Ctrl_Str
86         155         0.988162
           109         0.984090
           225         0.982522
dtype: float64 

Trial_Str  Ctrl_Str
88         40          0.970895
           26          0.958929
           72          0.954079
dtype: float64 



Based on the average of both features we found the following controls for each trial:
* Trial store 77: Store 233
* Trial store 86: Store 155
* Trial store 88: Store 40

### Assessment of trial

In [ ]:
## page 7